In [129]:
import numpy as np
import pandas as pd
import os

excelpath = 'excel_gaming_data'
#gamedf = pd.read_excel(excelpath + '/Tax10-11.xls')
ddict = {}
for filename in os.listdir(excelpath):
    f = os.path.join(excelpath, filename)
    # checking if it is a file
    if os.path.isfile(f) & ~filename.startswith('.'):
        ddict[filename[-6:-4]] = pd.read_excel(excelpath + '/' + filename)
        
        for idx, val in enumerate(ddict[filename[-6:-4]].iloc[0,:]):
            if idx > 1:
                ddict[filename[-6:-4]].columns.values[idx] = ddict[filename[-6:-4]].iloc[0,idx] + ' 20' + str(filename[-6:-4]) 
        if filename[-6:-4] != '11':
            ddict[filename[-6:-4]] = ddict[filename[-6:-4]].drop(columns=ddict[filename[-6:-4]].columns[0])
            ddict[filename[-6:-4]] = ddict[filename[-6:-4]].drop(columns=ddict[filename[-6:-4]].columns[0])
            
gamedf = ddict['11']
print(type(gamedf))
for ind in range(12,22):
    gamedf = pd.concat([gamedf,ddict[str(ind)]],axis=1)

gamedf.columns.values[0] = 'Location'
gamedf.columns.values[1] = 'Attributes'
print(gamedf.head())

<class 'pandas.core.frame.DataFrame'>
    Location     Attributes     JULY 2011   AUGUST 2011 SEPTEMBER 2011  \
0        NaN            NaN          JULY        AUGUST      SEPTEMBER   
1  Statewide    Slots Total         15224         15182          15189   
2  Statewide       Coins In  924881940.34  900224418.95   859887596.89   
3  Statewide            AGP   63485300.52    60548129.6    59638919.56   
4  Statewide  Avg Daily AGP    134.518715    128.650077     130.881821   

   OCTOBER 2011 NOVEMBER 2011 DECEMBER 2011  JANUARY 2011 FEBRUARY 2011  ...  \
0       OCTOBER      NOVEMBER      DECEMBER       JANUARY      FEBRUARY  ...   
1         15067         14572         14194         14367         14303  ...   
2  859014005.02  745499907.07  740211154.54  781224517.61  738530651.44  ...   
3   58207468.91   50949581.25    51507997.2   53065324.29   51589059.54  ...   
4    124.620713    116.751587    117.264377    119.248209    128.816781  ...   

   OCTOBER 2021 NOVEMBER 2021 DECEMB